## import libraries 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np

path = os.getcwd()
path

'/Users/rubenstuiver/code/rubenstuiver/olist/notebooks'

## Creating a variable csv_path that stores the path to data in the csv folder as a string

In [3]:
csv_path = '/Users/rubenstuiver/code/rubenstuiver/olist/raw_data/csv'
csv_path

'/Users/rubenstuiver/code/rubenstuiver/olist/raw_data/csv'

In [4]:
import pandas as pd
pd.read_csv(os.path.join(csv_path, 'olist_sellers_dataset.csv')).head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## Checking all the different csv files in the folder

In [5]:
os.listdir(csv_path)

['olist_sellers_dataset.csv',
 'product_category_name_translation.csv',
 'olist_orders_dataset.csv',
 '.gitkeep',
 'olist_order_items_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv']

In [6]:
len(os.listdir(csv_path))

10

In [7]:
file_names = [f for f in os.listdir(csv_path) if f.endswith('.csv')]
file_names

['olist_sellers_dataset.csv',
 'product_category_name_translation.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv']

In [8]:
len(file_names)

9

In [9]:
# file_names = str(file_names).replace("_dataset","")
# file_names = str(file_names).replace("olist_","")
# file_names = str(file_names).replace(".csv","")
# file_names

In [10]:
key_names = [key_name
    .replace('olist_','')
    .replace('_dataset','')
    .replace('.csv','')
    for key_name in file_names]
key_names

['sellers',
 'product_category_name_translation',
 'orders',
 'order_items',
 'customers',
 'geolocation',
 'order_payments',
 'order_reviews',
 'products']

In [11]:
## Creating the list of dict key key_names

In [12]:
data = {}
for key,file in zip(key_names, file_names):
    data[key] = pd.read_csv(os.path.join(csv_path, file))
data.keys()

dict_keys(['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'order_reviews', 'products'])

##  Implement the method get_data() in olist/data.py


In [13]:
os.getcwd()

'/Users/rubenstuiver/code/rubenstuiver/olist/notebooks'

In [14]:
from olist.data import Olist
Olist().get_data()['sellers'].head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## Exploratory Analysis

In [15]:
# import numpy as np
# import pandas as pd
# %load_ext autoreload
# %autoreload 2

In [16]:
# All data is still a dict
data = Olist().get_data()

All _id that are in the tables
- customer_id
- product_id
- seller_id
- review
- order_id

##  Run an automated exploratory analysis with pandas profiling

In [17]:
#install the pandas-profiling package
! pip install --quiet pandas-profiling

In [18]:
os.getcwd()

'/Users/rubenstuiver/code/rubenstuiver/olist/notebooks'

In [19]:
# And create a new "04-Decision-Science/reports" folder 
!mkdir -p ../raw_data/reports

In [20]:
import pandas_profiling
datasets_to_profile = ['orders', 'products', 'sellers',
                  'customers', 'order_reviews',
                  'order_items']

In [21]:
# for d in datasets_to_profile:
#     print('exporting: '+d)
#     profile = data[d].profile_report(title='Report for '+d)
#     profile.to_file(output_file="../raw_data/reports/"+d+'.html');

## Investigate the cardinalities of the various DataFrames (without using pandas profiling)

amount of unique: orders, reviews, sellers, products and customer

In [22]:
aggregates = np.array([
    [
        data["orders"].order_id.nunique(),
        data["order_reviews"].review_id.nunique(),
        data["sellers"].seller_id.nunique(),
        data["products"].product_id.nunique(),
        data["customers"].customer_id.nunique(),
    ],
    [
        data["orders"].order_id.count(),
        data["order_reviews"].review_id.count(),
        data["sellers"].seller_id.count(),
        data["products"].product_id.count(),
        data["customers"].customer_id.count(),
    ],
])

pd.DataFrame(data=np.vstack((aggregates, aggregates[0,:]/aggregates[1,:])).T,
             index=['orders', 'reviews', 'sellers', 'products', 'customers'],
            columns=['nunique', 'count','ratio'])

,nunique,count,ratio
orders,99441.0,99441.0,1.000000
reviews,98410.0,99224.0,0.991796
sellers,3095.0,3095.0,1.000000
products,32951.0,32951.0,1.000000
customers,99441.0,99441.0,1.000000


## amount of reviews per order

In [24]:
orders = data['orders'][['customer_id', 'order_id']]
reviews = data['order_reviews'][['order_id', 'review_id']]
items = data['order_items'][['order_id', 'product_id','seller_id']]

In [26]:
orders_reviews = orders.merge(reviews, on='order_id', how='outer')
orders_reviews.head(2)

,customer_id,order_id,review_id
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,a54f0611adc9ed256b57ede6b6eb5114
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,8d5266042046a06655c8db133d120ba5


In [27]:
# Check missing reviews
orders_reviews.isna().sum()

customer_id      0
order_id         0
review_id      768
dtype: int64

In [32]:
missing=orders_reviews.isna().sum()[2]

In [34]:
percent_missing_reviews_per_order=missing/len(reviews)*100
percent_missing_reviews_per_order

0.7740062888010965

In [78]:
orders_reviews.groupby(['order_id', 'customer_id']).count().hist()

array([[<AxesSubplot:title={'center':'review_id'}>]], dtype=object)